########## choose stock list (aka market) ##################

In [57]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date

In [58]:
## To acquire stock list in NASDAQ ###
ex1 = pd.read_csv('stock_info_us_1.csv')
ex2 = pd.read_csv('stock_info_us_2.csv')
df = pd.DataFrame(ex1['Symbol'].astype('str'))
df = pd.concat([df,pd.DataFrame(ex2['Symbol'].astype('str'))])
df = df.rename(columns={'Symbol' : 'ticker'})
df = df.reset_index(drop=True)
df = df.drop_duplicates()
df = df.reset_index(drop=True)

# df = df.loc[df['ticker'] == 'AMLX']
# df = df.reset_index(drop=True)
df

,ticker
0,AACG
1,AADI
2,AAGR
3,AAGRW
4,AAL
...,...
6755,ZTR
6756,ZTS
6757,ZUO
6758,ZVIA


### Acquire informations from yfinance function >> .info

In [59]:
# 1st pulling date for this data #
df['date_pulling'] = date.today()

In [60]:
# 2nd acquiring information from .info #
info_attribute_list = [
    'industry',
    'sector',
    'enterpriseValue',
    'totalCashPerShare'
]
# 3rd acquiring information from .quarterly_balance_sheet #
balancesheet_list = [
    'Total Assets',
    'Current Liabilities'
]

# 4th acquiring information from .quarterly_financials #
ttm = 4
financials_list = [
    'EBIT',
    'Operating Income'
]

info_attribute_list_buffer = []
balancesheet_list_buffer = []
financials_list_buffer = []
for i in range(len(df)):
    # print(i, " ",df['ticker'][i])
    yfticker = yf.Ticker(df['ticker'][i])

    # 2nd acquiring information from .info #
    info_attribute_list_buffer_r = []
    for j in range(len(info_attribute_list)):
        try:
            info_attribute_list_buffer_r.append(yfticker.info[info_attribute_list[j]])
        except:
            info_attribute_list_buffer_r.append(None)
    info_attribute_list_buffer.append(info_attribute_list_buffer_r)

    # 3rd acquiring information from .balancesheet #
    balancesheet_list_buffer_r = []
    for j in range(len(balancesheet_list)):
        try:
            balancesheet_list_buffer_r.append(yfticker.quarterly_balance_sheet.loc[yfticker.quarterly_balance_sheet.index == balancesheet_list[j]].values[0][0])
        except:
            balancesheet_list_buffer_r.append(None)
    balancesheet_list_buffer.append(balancesheet_list_buffer_r)

    # 4th acquiring information from .quarterly_financials #
    financials_list_buffer_r = []
    for j in range(len(financials_list)):
        try:
            financials_list_buffer_r.append(np.array([yfticker.quarterly_financials.loc[yfticker.quarterly_financials.index == financials_list[j]].values[0][i] for i in range(ttm)]).sum())
        except:
            financials_list_buffer_r.append(None)
    try:
        financials_list_buffer_r.append(yfticker.quarterly_financials.columns[0])
    except:
        financials_list_buffer_r.append(None)
    financials_list_buffer.append(financials_list_buffer_r)

financials_list.append('ttm_latest')
df = df.join(pd.DataFrame(info_attribute_list_buffer, columns=info_attribute_list))
df = df.join(pd.DataFrame(balancesheet_list_buffer, columns=balancesheet_list))
df = df.join(pd.DataFrame(financials_list_buffer, columns=financials_list))


#### Drop NAN #############
df = df.dropna()
##### Drop where marketcap and EBIT < 0#############
df = df.loc[df['enterpriseValue'] >0]
df = df.loc[df['EBIT'] >0]
df = df.loc[df['Operating Income'] >0]
df = df.reset_index(drop=True)

df.to_csv('data_stock_us.csv',index= False)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABR%5ED?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABR%5ED&crumb=oj8JRHy%2FYbf
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABR%5EE?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABR%5EE&crumb=oj8JRHy%2FYbf
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABR%5EF?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABR%5EF&crumb=oj8JRHy%2FYbf
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ACP%5EA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDet

############################################################################

In [61]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date
df = pd.read_csv('data_stock_us.csv')
print('sector', df['sector'].unique())
print('industry', df['industry'].unique())

sector ['Industrials' 'Technology' 'Financial Services' 'Consumer Cyclical'
 'Healthcare' 'Communication Services' 'Utilities' 'Consumer Defensive'
 'Energy' 'Basic Materials' 'Real Estate']
industry ['Airlines' 'Building Products & Equipment' 'Consumer Electronics'
 'Insurance - Life' 'Travel Services' 'Biotechnology'
 'Medical Care Facilities' 'Software - Infrastructure'
 'Semiconductor Equipment & Materials' 'Business Equipment & Supplies'
 'Software - Application' 'Semiconductors' 'Electrical Equipment & Parts'
 'Entertainment' 'Utilities - Regulated Electric'
 'Education & Training Services' 'Conglomerates'
 'Medical Instruments & Supplies'
 'Drug Manufacturers - Specialty & Generic' 'Electronic Components'
 'Computer Hardware' 'Drug Manufacturers - General' 'Capital Markets'
 'Furnishings, Fixtures & Appliances' 'Internet Retail'
 'Food Distribution' 'Internet Content & Information' 'Oil & Gas E&P'
 'Trucking' 'Home Improvement Retail' 'Specialty Retail' 'Thermal Coal'
 'Solar' '

In [62]:

### Next I perform calculations for MF_ROC and MF_EY ###
represents_earning = 'Operating Income'
# represents_earning = 'EBIT'

df['MF_ROC'] = df[represents_earning]/(df['Total Assets'] - df['Current Liabilities'])
df['MF_EY'] = df[represents_earning]/df['enterpriseValue']
df

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY
0,AAL,2024-08-18,Airlines,Industrials,3.705987e+10,13.685,6.412500e+10,2.461000e+10,1.996000e+09,2.852000e+09,2024-06-30,0.072175,0.076957
1,AAON,2024-08-18,Building Products & Equipment,Industrials,7.222197e+09,0.035,1.010716e+09,1.413190e+08,2.431950e+08,2.426760e+08,2024-06-30,0.279131,0.033601
2,AAPL,2024-08-18,Consumer Electronics,Technology,3.476398e+12,4.065,3.316120e+11,1.316240e+11,1.216250e+11,1.205940e+11,2024-06-30,0.603006,0.034689
3,ABL,2024-08-18,Insurance - Life,Financial Services,8.722294e+08,1.277,4.838549e+08,5.538306e+07,1.505907e+07,1.895698e+07,2024-06-30,0.044243,0.021734
4,ABNB,2024-08-18,Travel Services,Consumer Cyclical,6.458017e+10,17.790,2.632000e+10,1.582800e+10,1.971000e+09,1.598000e+09,2024-06-30,0.152307,0.024744
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2133,ZBH,2024-08-18,Medical Devices,Healthcare,2.815703e+10,2.063,2.152000e+10,3.608400e+09,1.224200e+09,1.507600e+09,2024-06-30,0.084169,0.053543
2134,ZIP,2024-08-18,Staffing & Employment Services,Industrials,8.540356e+08,5.268,6.652730e+08,7.577100e+07,7.541800e+07,5.447000e+07,2024-06-30,0.092400,0.063780
2135,ZTO,2024-08-18,Integrated Freight & Logistics,Industrials,1.258625e+10,24.325,8.987536e+10,2.326055e+10,1.087243e+10,1.032418e+10,2024-03-31,0.154983,0.820275
2136,ZTS,2024-08-18,Drug Manufacturers—Specialty & Generic,Healthcare,8.842400e+10,3.474,1.416500e+10,1.797000e+09,3.129000e+09,3.214000e+09,2024-06-30,0.259864,0.036348


In [63]:
### Drop Utilities, Energy, and Financial Services  as suggested from the book ###
sectortoexclude = [
    'Utilities',
    'Energy',
    'Financial Services',
    'Real Estate'
]
for i in sectortoexclude:
    print('exclude sector > ',i)
    try:
         
        df = df.loc[df['sector'] != i]
    except:
        None
        
industrytoexclude =[
    'Engineering & Construction',
    'Building Products & Equipment',
    'Building Materials'
]
for i in industrytoexclude:
    print('exclude industry > ',i)
    try:
         
        df = df.loc[df['industry'] != i]
    except:
        None

df = df.reset_index(drop=True)

exclude sector >  Utilities
exclude sector >  Energy
exclude sector >  Financial Services
exclude sector >  Real Estate
exclude industry >  Engineering & Construction
exclude industry >  Building Products & Equipment
exclude industry >  Building Materials


In [64]:
#### Select market size by choosing ' market ' ###
market =  50000000### in USD
df_market = df.loc[df['enterpriseValue'] >= market]
df_market = df_market.reset_index(drop=True)

In [65]:
### Ranking regarding MFs####

df_market['Ranking_MF_ROC'] = df_market['MF_ROC'].rank()
df_market['Ranking_MF_EY'] = df_market['MF_EY'].rank()
df_market['Ranking_MF'] = df_market['Ranking_MF_ROC'] + df_market['Ranking_MF_EY']

df_market.loc[df_market['Ranking_MF'] == df_market['Ranking_MF'].max()]

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
559,RMNI,2024-08-18,Software—Application,Technology,113291000.0,1.48,367369000.0,303236000.0,30009000.0,39537000.0,2024-06-30,0.616484,0.348986,1508.0,1500.0,3008.0


In [66]:
numstocks = 50
df_sorted = df_market.sort_values(by=['Ranking_MF'],ascending=False)
df_sorted = df_sorted.reset_index(drop=True)
df_sorted[:numstocks]

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
0,RMNI,2024-08-18,Software—Application,Technology,1.132910e+08,1.480,3.673690e+08,3.032360e+08,3.000900e+07,3.953700e+07,2024-06-30,0.616484,0.348986,1508.0,1500.0,3008.0
1,JFIN,2024-08-18,Internet Content & Information,Communication Services,7.779670e+08,2.678,5.160597e+09,2.004158e+09,1.299660e+09,1.299660e+09,2024-03-31,0.411749,1.670585,1483.0,1518.0,3001.0
2,NVO,2024-08-18,Biotechnology,Healthcare,5.957051e+11,14.055,3.693830e+11,1.879090e+11,1.166360e+11,1.114590e+11,2024-06-30,0.614187,0.187104,1506.0,1460.0,2966.0
3,KARO,2024-08-18,Software—Application,Technology,4.906297e+08,32.242,4.565444e+09,1.055226e+09,1.155279e+09,1.117875e+09,2024-05-31,0.318463,2.278449,1433.0,1519.0,2952.0
4,BWMX,2024-08-18,Specialty Retail,Consumer Cyclical,5.444902e+09,11.364,1.101829e+10,3.860136e+09,2.397458e+09,2.377988e+09,2024-06-30,0.332207,0.436737,1443.0,1503.0,2946.0
5,SIGA,2024-08-18,Drug Manufacturers—Specialty & Generic,Healthcare,6.433619e+08,1.499,1.926586e+08,2.137805e+07,1.027231e+08,1.027231e+08,2024-06-30,0.599736,0.159666,1504.0,1437.0,2941.0
6,OMAB,2024-08-18,Airports & Air Services,Industrials,3.498960e+10,4.169,2.590749e+10,4.752860e+09,7.834555e+09,7.527290e+09,2024-06-30,0.355822,0.215129,1462.0,1473.0,2935.0
7,GTX,2024-08-18,Auto Parts,Consumer Cyclical,3.184157e+09,0.448,2.229000e+09,1.274000e+09,5.040000e+08,4.690000e+08,2024-06-30,0.491099,0.147292,1494.0,1423.0,2917.0
8,TH,2024-08-18,Specialty Business Services,Industrials,1.057719e+09,1.536,6.970490e+08,2.355060e+08,1.823560e+08,1.738810e+08,2024-06-30,0.376738,0.164392,1471.0,1441.0,2912.0
9,WLKP,2024-08-18,Chemicals,Basic Materials,1.448592e+09,1.873,1.288362e+09,4.960700e+07,3.727280e+08,3.677860e+08,2024-06-30,0.296900,0.253892,1417.0,1486.0,2903.0


In [67]:
df_sorted_2 = df_sorted[:numstocks].copy()
df_sorted_2 = df_sorted_2.sort_values(by=['totalCashPerShare'],ascending=False)
df_sorted_2 = df_sorted_2.reset_index(drop=True)
df_sorted_2

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
0,TLK,2024-08-18,Telecom Services,Communication Services,6.996729e+13,283.892,2.859920e+14,8.317300e+13,4.434200e+13,4.457200e+13,2024-06-30,0.219762,0.637040,1303.0,1509.0,2812.0
1,TSM,2024-08-18,Semiconductors,Technology,3.534018e+12,79.010,5.787891e+12,1.026180e+12,1.082195e+12,1.023834e+12,2024-06-30,0.215014,0.289708,1294.0,1491.0,2785.0
2,DDS,2024-08-18,Department Stores,Consumer Cyclical,5.212588e+09,66.074,3.937156e+09,1.130290e+09,9.238860e+08,8.957790e+08,2024-04-30,0.319138,0.171849,1434.0,1444.0,2878.0
3,ASR,2024-08-18,Airports & Air Services,Industrials,8.908710e+10,49.990,7.299319e+10,4.898742e+09,1.674405e+10,1.580910e+10,2024-03-31,0.232164,0.177457,1324.0,1454.0,2778.0
4,PDD,2024-08-18,Internet Retail,Consumer Cyclical,5.898885e+11,43.947,3.778261e+11,1.513980e+11,9.543905e+10,7.774340e+10,2024-03-31,0.343347,0.131793,1453.0,1401.0,2854.0
5,KARO,2024-08-18,Software—Application,Technology,4.906297e+08,32.242,4.565444e+09,1.055226e+09,1.155279e+09,1.117875e+09,2024-05-31,0.318463,2.278449,1433.0,1519.0,2952.0
6,AMR,2024-08-18,Coking Coal,Basic Materials,2.803547e+09,25.826,2.492952e+09,3.086540e+08,5.282050e+08,5.430350e+08,2024-06-30,0.248608,0.193696,1347.0,1466.0,2813.0
7,STNE,2024-08-18,Software—Infrastructure,Technology,8.895593e+09,15.831,5.269974e+10,3.004820e+10,3.178735e+09,5.807946e+09,2024-06-30,0.256404,0.652901,1362.0,1510.0,2872.0
8,NVO,2024-08-18,Biotechnology,Healthcare,5.957051e+11,14.055,3.693830e+11,1.879090e+11,1.166360e+11,1.114590e+11,2024-06-30,0.614187,0.187104,1506.0,1460.0,2966.0
9,BWMX,2024-08-18,Specialty Retail,Consumer Cyclical,5.444902e+09,11.364,1.101829e+10,3.860136e+09,2.397458e+09,2.377988e+09,2024-06-30,0.332207,0.436737,1443.0,1503.0,2946.0
